## 05.1 Creating an NFT
##### Peter Gruber, Mattia Biancaterra (mattia.biancaterra@usi.ch, peter.gruber@usi.ch)
2023-01-19

* Simplified NFT

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
import base64
import IPython.display
import hashlib, requests

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()['last-round']

39064362

In [5]:
print(MyAlgo['public'])

WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI


## Showcase

this is an example of NFT and how it looks like on Algorand block explorers

- https://explorer.perawallet.app/assets/1038214068/

## Create a simplified NFT
* In theory, NFTs require extensive metadata and hash information for integrity
* A simplified NFT with only an image link *currently* works with the Pera Algo Wallet
    * A first step for learning
    * **Not recommended**

#### Step 1: the link to the image
* Link is on the Pinata gateway to IPFS
* Instead of filename, use `cid` = *content identifier*
* Test the link (*very slow*)

In [6]:
cid = "QmXEKbYJHKVbZ9ZAVMJKrX12koZ3bvMSzXuVLyrfULoV89"
url = 'https://gateway.pinata.cloud/ipfs/'+cid
print(url)

https://gateway.pinata.cloud/ipfs/QmXEKbYJHKVbZ9ZAVMJKrX12koZ3bvMSzXuVLyrfULoV89


#### Step 2: the NFT transaction
* NFT is based on an ASA
    * `total` (supply) = 1
    * Additional field `url`

##### Step 2.1a: Setup

In [7]:
NFT_name = "Lugano NFT"
NFT_unit = "LUG1"
NFT_supply = 1
NFT_decimals = 0
# plus the URL from above

##### Step 2.1b: Same `AssetConfigTxn` as for creating a token
* `MyAlgo` creates mints the NFT

In [8]:
sp=algod_client.suggested_params()

txn = AssetConfigTxn(
    sender=MyAlgo['public'],
    sp=sp,
    total=NFT_supply,
    decimals=NFT_decimals,
    asset_name=NFT_name,
    unit_name=NFT_unit,
    manager=MyAlgo['public'],
    reserve=MyAlgo['public'],
    freeze=MyAlgo['public'],
    clawback=MyAlgo['public'],
    url=url                            # Direct link to file, no metadata
)

#### Step 3: Sign and send

In [9]:
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

3CRIJQTFEKQRFXGJRMQMX3GLUV22COKA3TEIRRB3EU6ILN4I3HKA


#### Step 4: Wait for confirmation

In [10]:
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39064368.
Waiting for round 39064368 to finish.
Waiting for round 39064369 to finish.
Transaction 3CRIJQTFEKQRFXGJRMQMX3GLUV22COKA3TEIRRB3EU6ILN4I3HKA confirmed in round 39064370.


#### Step 5: Verification
##### Step 5.1: NFT index
* The NFT `index` is automatically created

In [11]:
NFT_id = txinfo['asset-index']
print(NFT_id)

642547439


##### Step 5.2: Inspect on Pera Explorer and in Pera Wallet
* Inspect on Pera Explorer
* Also inspect in the Pera Wallet (TestNet mode)

In [14]:
print(cred['explore_test']+'asset/{}'.format(NFT_id))

https://testnet.explorer.perawallet.app/asset/642547439


##### Step 6: Check on Blockchain
* NFT is an asset that we hold (like USDC)
* But also an Asset that we have created

In [15]:
asset_holdings_df(algod_client,MyAlgo['public'])

,amount,unit,asset-id,name,decimals
0,37.938357,ALGO,0,Algorand,6
1,967.000000,PWSC,592359981,Peters WSC coin,2
2,1.000000,LUG1,592373389,Lugano NFT,0
3,1000.000000,WSC,640596559,Peters WSC coin,2
4,1000.000000,WSC,641958950,Peters WSC coin,2
5,1000.000000,WSC,641960001,Peters WSC coin,2
6,75.000000,TEMP,641963975,Peters Tempcoin,1
7,980.000000,WSC,642000039,Peters WSC coin,2
8,1.000000,LUG1,642547439,Lugano NFT,0


In [16]:
# Looking for assets that we hold
[asset for asset in algod_client.account_info(MyAlgo['public'])['assets'] if asset['asset-id']==NFT_id]

[{'amount': 1, 'asset-id': 642547439, 'is-frozen': False}]

In [17]:
# Looking for asset that we have created
# NOTE: slightly different naming!!
[asset for asset in algod_client.account_info(MyAlgo['public'])['created-assets'] if asset['index']==NFT_id]

[{'index': 642547439,
  'params': {'clawback': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'creator': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'decimals': 0,
   'default-frozen': False,
   'freeze': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'manager': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'name': 'Lugano NFT',
   'name-b64': 'THVnYW5vIE5GVA==',
   'reserve': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'total': 1,
   'unit-name': 'LUG1',
   'unit-name-b64': 'TFVHMQ==',
   'url': 'https://gateway.pinata.cloud/ipfs/QmXEKbYJHKVbZ9ZAVMJKrX12koZ3bvMSzXuVLyrfULoV89',
   'url-b64': 'aHR0cHM6Ly9nYXRld2F5LnBpbmF0YS5jbG91ZC9pcGZzL1FtWEVLYllKSEtWYlo5WkFWTUpLclgxMmtvWjNidk1Telh1Vkx5cmZVTG9WODk='}}]

## Exercise
* Repeat the entire notebook with a different file format, e.g. `png` or `pdf`
